## But:
rechercher les informations de chansons dans la section "Credits":
- tags (Pop, rock, etc. )
- crédits : compositeur, producteur, etc.

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import os
import requests
import json

from bs4 import BeautifulSoup

In [2]:
import re

In [3]:
url = "https://genius.com/Simon-and-garfunkel-bridge-over-troubled-water-lyrics"

In [4]:
# requête de connection
url_req = requests.get(url)
print(type(url_req))
print(url_req)

print('-'*30)
print('-'*30)


<class 'requests.models.Response'>
<Response [200]>
------------------------------
------------------------------


In [5]:
# transformer le texte en objet 'soup',qui permet d'utilisr les fonctionallités de 'BeautifulSoup'

soup = BeautifulSoup(url_req.content, 'html.parser')

In [6]:
#soup

In [37]:
S1 = soup.css.select('div',{"class":"SongTags_Container_"},)

In [38]:
S1

[<div id="application"><svg aria-hidden="true" style="position:absolute;width:0;height:0;overflow:hidden"><defs><lineargradient id="appleMusicGradient" x1="0" x2="0" y1="1" y2="0"><stop stop-color="#FA233B"></stop><stop offset="1" stop-color="#FB5C74"></stop></lineargradient></defs></svg><div class="sc-161f4128-0 cGgzqo sc-c6a1ca7-0 bCLuZv"><div class="sc-161f4128-1 csfPYh"><div class="sc-f5fa7f9a-0 JJLsK"><div id="div-gpt-ad-desktop_song_combined_leaderboard-desktop_song_combined_leaderboard-1"></div><div class="sc-f5fa7f9a-1 lkTyXJ"></div></div></div></div><nav class="StickyNav-desktop__Container-sc-72d92c60-0 hTbEFe" id="sticky-nav"><div class="StickyNav-desktop__Left-sc-72d92c60-1 fAkBkG"><a class="PageHeaderLogo__Link-sc-c56fc0b0-0 clUGTX StickyNav-desktop__Logo-sc-72d92c60-7 kHFFiu" href="https://genius.com"><svg aria-label="Genius Logo" viewbox="0 0 100 15" xmlns="http://www.w3.org/2000/svg"><path d="M11.7 2.9s0-.1 0 0c-.8-.8-1.7-1.2-2.8-1.2-1.1 0-2.1.4-2.8 1.1-.2.2-.3.4-.5.6v.1

In [9]:
# soup

In [10]:
url_req.status_code

200

In [11]:
cred = soup.find(string="Credits")

In [12]:
container = cred.find_parents(limit=2)[1]

In [13]:
lst_cred = container.find_all(name="div",attrs={"class":re.compile("^SongInfo__Credit")})

In [14]:
lst_cred[0].find(name="div").text


'Producers'

In [15]:
lst_cred[0].find(name="a").text

'Art Garfunkel'

In [16]:
DICT = {}
for el in lst_cred:
    try:
        DICT[el.find(name="div").text] = el.find(name="a").text
    except:
        pass


In [17]:
list(DICT.keys())[:5]

['Producers', 'Writer', 'Mastered At', 'Manufacturer', 'Copyright ©']

In [18]:
tags = soup.find(string="Tags")

In [19]:
tags_container = tags.find_parents(limit=2)[1]

In [20]:
# tags_container

In [21]:
all_tags = tags_container.find_all(name="a",attrs={"class":re.compile("^SongTags")})

In [22]:
all_tags

[<a class="SongTags__Tag-sc-db268d15-2 eDJmdO" href="https://genius.com/tags/pop">Pop</a>,
 <a class="SongTags__Tag-sc-db268d15-2 cYGtri" href="https://genius.com/tags/folk-pop">Folk Pop</a>,
 <a class="SongTags__Tag-sc-db268d15-2 cYGtri" href="https://genius.com/tags/in-english">In English</a>,
 <a class="SongTags__Tag-sc-db268d15-2 cYGtri" href="https://genius.com/tags/progressive-pop">Progressive Pop</a>,
 <a class="SongTags__Tag-sc-db268d15-2 cYGtri" href="https://genius.com/tags/adult-contemporary">Adult Contemporary</a>,
 <a class="SongTags__Tag-sc-db268d15-2 cYGtri" href="https://genius.com/tags/ballad">Ballad</a>,
 <a class="SongTags__Tag-sc-db268d15-2 cYGtri" href="https://genius.com/tags/baroque-pop">Baroque Pop</a>,
 <a class="SongTags__Tag-sc-db268d15-2 cYGtri" href="https://genius.com/tags/gospel">Gospel</a>]

In [23]:
LST_TAGS = [el.text for el in all_tags]
LST_TAGS

['Pop',
 'Folk Pop',
 'In English',
 'Progressive Pop',
 'Adult Contemporary',
 'Ballad',
 'Baroque Pop',
 'Gospel']

In [26]:
def process_url(url:str)->tuple:
    """returns [CODE], [DICT], [LIST] , [DICT]

        with 
            CODE = html page code: 
                200: data could be fetched and processed
                else: no data, no processing
                
            DICT = dictionary with credits with following structure
                [role]:[list of names]
            
            LIST = list of tags of music
            
            DICT = dictionary with errors (if any) during processing
    
    """
    ### go to url
    url_req = requests.get(url)
    ERR = {"creds":[], "tags":[]}

    
    if url_req.status_code !=200: ## 200 = OK
        return url_req.status_code, {}, [], ERR

    

    soup = BeautifulSoup(url_req.content, 'html.parser')

    
    ### FIND information of Credits
    lst_cred = []
    try:
        cred = soup.find(string="Credits")
        container = cred.find_parents(limit=2)[1]
        lst_cred = container.find_all(name="div",attrs={"class":re.compile("^SongInfo__Credit")})
        print("creds found")
    except Exception as err:
        ERR["creds"].append("error in finding Credits")
        ERR["creds"].append(err)

    
    
    DICT_CRED = {}
    for el in lst_cred:
        key=None
        value=None
        try:
            key = el.find(name="div").text
            try:
                value = el.find_all(name="div")[1].text ## simple text
            except:
                value = [item.text for item in el.find_all(name="a")]  ## sub-elements
            
            DICT_CRED[key] = value
        except Exception as err:
            ERR["creds"].append((key,value))
            ERR["creds"].append(err)

    ### FIND tags
    all_tags = []
    try:
        tags = soup.find(string="Tags")
        tags_container = tags.find_parents(limit=2)[1]
        all_tags = tags_container.find_all(name="a",attrs={"class":re.compile("^SongTags")})

        print("tags found")
    except Exception as err:
        ERR["tags"].append("error in finding Tags")
        ERR["tags"].append(err)
    
    LST_TAGS = []
    for el in all_tags:
        try:
            LST_TAGS.append(el.text)
        except Exception as err:
            ERR["tags"].append(err)
    
    return url_req.status_code, DICT_CRED, LST_TAGS, ERR

In [27]:
url2 = "https://genius.com/Sza-low-lyrics"

code, creds, tags, errs = process_url(url=url2)

creds found
tags found


In [28]:
code

200

In [29]:
tags

['R&B',
 'Rap',
 'Neo Soul',
 'In English',
 'USA',
 'Contemporary R&B',
 'Pop Rap',
 'Trap']

In [30]:
creds

{'Producers': 'Alessandro Buccellati, Rob Bisel & Aire Atlantica',
 'Writers': 'SZA, Jozzy, Rob Bisel, Alessandro Buccellati, Aire Atlantica, Travis Scott & Baby Shad',
 'Engineered At': 'Westlake Studios',
 'Mixed At': 'The Gift Shop',
 'Assistant Recording Engineer': 'Robert N. Johnson',
 'Recording Engineer': '206Derek',
 'Publisher': 'Top Dawg Entertainment, SZA, Universal Music Group, Sony Songs, Bisel Creative Sound, Sony Music Entertainment, Two Many Songs, AprilMayJune Music, Songs by MPA & The Unorthodox Elite',
 'Mastered At': 'Becker Mastering, Pasadena, CA',
 'Assistant Mastering Engineer': 'Katie Harvey & Noah McCorkle',
 'Drum Programmer': 'Aire Atlantica',
 'Video Director': 'Gibson Hazard',
 'Mastering Engineer': 'Dale Becker',
 'Mixing Engineer': 'Jon Castelli, 206Derek & Joshua Deguzman',
 'Engineer': 'Rob Bisel',
 'Keyboards': 'Rob Bisel & Alessandro Buccellati',
 'Accordion': 'Alessandro Buccellati',
 'Additional Vocals': 'Rob Bisel, Alessandro Buccellati & Travis S

In [31]:
errs

{'creds': [], 'tags': []}

In [32]:
url3 = "https://genius.com/Taylor-swift-loml-lyrics"

code, creds, tags, errs = process_url(url=url3)

creds found
tags found


In [33]:
tags

['Pop',
 'Singer-Songwriter',
 'Chamber Pop',
 'Easy Listening',
 'Ballad',
 'USA',
 'In English']

In [34]:
creds

{'Producers': 'Aaron Dessner & Taylor Swift',
 'Writers': 'Taylor Swift & Aaron Dessner',
 'Mastered At': 'Sterling Sound',
 'Mixed At': 'MixStar Studios',
 'Publisher': 'TASRM PUBLISHING, Universal Music Group, Ingrid Stella Music & Sony Music Entertainment',
 'Recording Arranger': 'Rob Moose',
 'Mixing Engineer': 'Bryce Bordone & Șerban Ghenea',
 'Bass Synthesizer': 'Aaron Dessner',
 'Mastering Engineer': 'Randy Merrill',
 'Recording Engineer': 'Laura Sisk, Bella Blasko, Christopher Rowe & Jonathan Low',
 'Percussion': 'Glenn Kotche',
 'Drums': 'Glenn Kotche',
 'Synthesizer': 'Aaron Dessner',
 'Keyboards': 'Aaron Dessner',
 'Piano': 'Aaron Dessner',
 'Label': 'Republic Records',
 'Phonographic Copyright ℗': 'Taylor Swift',
 'Copyright ©': 'Taylor Swift',
 'Recorded At': 'Long Pond (Hudson Valley, NY) / Narwhal Studios (Chicago, IL) / Smilo Sound (Orcas Island, WA) / Electric Lady Studios (New York, NY) / Prime Recording (Nashville, TN)',
 'Released on': 'April 19, 2024',
 'Songs That